In [2]:
def _pair_exp_cov(X, Y, span= 10):
    """
    Calculate the exponential covariance between two timeseries of returns.
    :param X: first time series of returns
    :type X: pd.Series
    :param Y: second time series of returns
    :type Y: pd.Series
    :param span: the span of the exponential weighting function, defaults to 180
    :type span: int, optional
    :return: the exponential covariance between X and Y
    :rtype: float
    """
    covariation = (X - X.mean()) * (Y - Y.mean())
    # Exponentially weight the covariation and take the mean
    if span < 3:
        warnings.warn("it is recommended to use a higher span, e.g 30 days")
    return covariation.ewm(span=span).mean().iloc[-1]

def exp_cov(
    return_data, span = 3, frequency = 25, log_returns = False, **kwargs
):
    """
    Estimate the exponentially-weighted covariance matrix, which gives
    greater weight to more recent data.
    :param prices: adjusted closing prices of the asset, each row is a date
                   and each column is a ticker/id.
    :type prices: pd.DataFrame
    :param returns_data: if true, the first argument is returns instead of prices.
    :type returns_data: bool, defaults to False.
    :param span: the span of the exponential weighting function, defaults to 180
    :type span: int, optional
    :param frequency: number of time periods in a year, defaults to 252 (the number
                      of trading days in a year)
    :type frequency: int, optional
    :param log_returns: whether to compute using log returns
    :type log_returns: bool, defaults to False
    :return: annualised estimate of exponential covariance matrix
    :rtype: pd.DataFrame
    """
    if not isinstance(return_data, pd.DataFrame):
        warnings.warn("data is not in a dataframe", RuntimeWarning)
        prices = pd.DataFrame(return_)
        
    assets = prices.columns
    if returns_data:
        returns = prices
    else:
        returns = returns_from_prices(prices, log_returns)
    N = len(assets)

    # Loop over matrix, filling entries with the pairwise exp cov
    S = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            S[i, j] = S[j, i] = _pair_exp_cov(
                returns.iloc[:, i], returns.iloc[:, j], span
            )
    cov = pd.DataFrame(S * frequency, columns=assets, index=assets)

    return cov 

In [2]:
import pandas as pd
import numpy as np
from sklearn.covariance import EmpiricalCovariance
from scipy.optimize import minimize
import warnings
import pyRMT
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


class TAQCovariance(object):
    def __init__(self):
        self.return_matrix = pd.read_csv("5_mins_mid_quote_ret.csv")
        self.return_matrix["Datetime"] = pd.to_datetime(self.return_matrix.Datetime)
        
        
        self.dates = self.return_matrix["Datetime"].apply(lambda x: x.strftime("%Y%m%d")).unique()
        self.return_matrix.set_index("Datetime",inplace=True)
        #self.return_matrix = self.return_matrix.loc[pd.to_datetime(self.return_matrix.index.strftime("%H:%M")) >= pd.to_datetime("13:35")]
        
        self.pre_processing()
        
    def pre_processing(self):
        
        # subtract_mean
        self.return_matrix -= self.return_matrix.mean()
        # cross-sectional daily volatility
        cross_sectional_vol = (self.return_matrix ** 2).sum(axis=1)
        cross_sectional_vol = np.sqrt(cross_sectional_vol)
        
        
        def pre_process(x):
            # normalize by cross_sectional vol
            x = x / cross_sectional_vol
            return x

        self.return_matrix = self.return_matrix.apply(lambda x: pre_process(x))
        return 
    
    def normalize_train(self, start_date, end_date):
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        sub_df = self.return_matrix.copy()
        sub_df = sub_df.loc[(sub_df.index >= start_date) & (sub_df.index < end_date)]
        
        cross_time_vol = []
        def normalize(x):
            # normalize by cross_time vol
            std = x.std()
            cross_time_vol.append(std)
            x = x / std
            return x
        
        sub_df = sub_df.apply(lambda x: normalize(x))
        return sub_df,np.array(cross_time_vol)
        
        
    # Compute strategy weights  
    def compute_weights(self,return_matrix, cov_matrix, index, weight_method):
        num = cov_matrix.shape[0]
        cov_matrix_inv = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
        if weight_method == 'historical':
            g_list = np.array([1] * num).reshape(-1,1)

            w = cov_matrix_inv.dot(g_list)/np.dot(g_list.T,np.dot(cov_matrix_inv,g_list))
            return w

        if weight_method == 'omniscient':
            g_list = np.sqrt(num) * return_matrix.iloc[0]
            g_list = g_list.values.reshape(-1,1)

            w = cov_matrix_inv.dot(g_list)/np.dot(g_list.T,np.dot(cov_matrix_inv,g_list))
            return w
        
        if weight_method == "mean_reverting":
            
            test_start_date = pd.to_datetime(self.dates[index])
            train_end_date = pd.to_datetime(self.dates[index - 1])
            prev_day_return = self.return_matrix.loc[(self.return_matrix.index >= train_end_date)\
                                                     & (self.return_matrix.index < test_start_date)]
            g_list = np.sqrt(num) * prev_day_return.iloc[-1]
            g_list = g_list.values.reshape(-1,1)

            w = cov_matrix_inv.dot(g_list)/np.dot(g_list.T,np.dot(cov_matrix_inv,g_list))
            return w
        
        if weight_method == 'random_long_short':
            g_list = np.random.uniform(0, 1, num)
            g_list = g_list.reshape(-1,1)

            w = cov_matrix_inv.dot(g_list)/np.dot(g_list.T,np.dot(cov_matrix_inv,g_list))
            return w
    
    
    def compute_return_vol(self, w, return_matrix):
        w = w.values
        total_return = np.dot(return_matrix, w)
        total_return_square = total_return ** 2
        R_square = total_return_square.sum()/return_matrix.shape[0]
        return R_square
        
    def compute_biastats(self, w, return_matrix):
        w = w.values
        total_return = np.dot(return_matrix, w)
        total_return_square = total_return ** 2 
        R_square = total_return_square.sum()/return_matrix.shape[0]
        std_var = np.sqrt(R_square)
        bias_stats = (total_return / std_var).std()
        return bias_stats 
        
    def rolling_backtest(self, train_win=7, test_win=2, shrinkage_method='Empirical', weight_method = "historical"):
        
        R_square_list = []
        Bias_stats_list = []
        for i in range(len(self.dates) - (train_win + test_win)):
            
            train_start_date = self.dates[i]
            train_end_date = self.dates[i + train_win]
            
            test_start_date = self.dates[i + train_win]
            test_end_date = self.dates[i + train_win + test_win]
            
            train_df, cross_time_vol = self.normalize_train(train_start_date,train_end_date)
            test_return = self.return_matrix.loc[(self.return_matrix.index >= test_start_date) & (self.return_matrix.index < test_end_date)]

            if shrinkage_method == 'Empirical':
                train_cov = EmpiricalCovariance().fit(train_df)
                train_cov = pd.DataFrame(train_cov.covariance_, index=train_df.columns, columns=train_df.columns)

            if shrinkage_method == 'Clipped':
                train_cov = pyRMT.clipped(train_df, return_covariance=True)
                train_cov = pd.DataFrame(train_cov, index=train_df.columns, columns=train_df.columns)
            

            if shrinkage_method == 'Optimal':
                train_cov = pyRMT.optimalShrinkage(train_df, return_covariance=True)
                train_cov = pd.DataFrame(train_cov, index=train_df.columns, columns=train_df.columns)
                
            # restoring volatility to our correlation matrix
            cross_time_vol_matrix = cross_time_vol.reshape(-1,1) * cross_time_vol
            train_cov = train_cov * cross_time_vol_matrix
            
            weights = self.compute_weights(test_return,train_cov,i+train_win,weight_method)
            R_square = self.compute_return_vol(weights,test_return)
            Bias_stats = self.compute_biastats(weights, test_return)
            R_square_list.append(R_square)
            Bias_stats_list.append(Bias_stats)
            
        return R_square_list, Bias_stats_list 
        
    def generate_R_square_table(self, train_win, test_win):
        shrinkage_methods = ['Empirical',"Clipped","Optimal"]
        weight_methods = ["historical",'omniscient', "mean_reverting",'random_long_short']
        table_dic_R_squared = {}
        table_dic_Bias = {}
        for shrinkage_method in shrinkage_methods:
            temp_dic_R_squared = {}
            temp_dic_Bias = {}
            for weight_method in weight_methods:
                R_square_list, Bias_stats_list = self.rolling_backtest(train_win,test_win,shrinkage_method,weight_method)
                mean_R_square = np.mean(R_square_list)
                mean_Bias = np.mean(Bias_stats_list)
                
                temp_dic_R_squared[weight_method] = mean_R_square
                temp_dic_Bias[weight_method] = mean_Bias - 1
  
            table_dic_R_squared[shrinkage_method] = temp_dic_R_squared
            table_dic_Bias[shrinkage_method] = temp_dic_Bias
        return pd.DataFrame(table_dic_R_squared), pd.DataFrame(table_dic_Bias)


In [34]:
TAQCovariance_obj = TAQCovariance()
TAQCovariance_obj.return_matrix.ABT
TAQCovariance_obj.dates

array(['20070904', '20070905', '20070906', '20070907', '20070910',
       '20070911', '20070912', '20070913', '20070914', '20070917',
       '20070918', '20070919', '20070920'], dtype=object)

In [15]:
TAQCovariance_obj.rolling_backtest(shrinkage_method="Optimal",weight_method = "omniscient")

([0.0815034049078314,
  0.0827666916613883,
  0.08218748200164708,
  0.07992880347675381],
 [0.998733940899264,
  0.9999998071264448,
  0.9992903319277684,
  0.9985099169910595])

In [35]:
TAQCovariance_obj.generate_R_square_table(10,2)

(                   Empirical   Clipped   Optimal
 historical          0.083275  0.082422  0.082417
 omniscient          0.080268  0.080711  0.079290
 mean_reverting      0.055777  0.055544  0.055398
 random_long_short   0.218167  0.126026  1.885443,
                       Empirical   Clipped   Optimal
 historical        -1.226363e-04 -0.000078 -0.000062
 omniscient        -1.572318e-03 -0.001397 -0.001339
 mean_reverting    -4.958163e-04 -0.000155 -0.000112
 random_long_short -3.911632e-07 -0.000002 -0.002790)

In [174]:
# Seperating training and testing dataset on rolling basis
    

In [4]:
np.array([1] * 3).reshape(-1,1)

array([[1],
       [1],
       [1]])

In [3]:
import pandas as pd

In [52]:
a = {"1": pd.DataFrame([1,2,3,4,5],index=[1,2,3,4,5],columns = ["a"]), "2": pd.DataFrame([1,2,3,4],index=[1,2,3,4], columns = ["b"])}

In [56]:
v = pd.DataFrame([1,2,3,4,5],index=[1,2,3,4,5],columns = ["a"])

In [58]:
l = pd.DataFrame([])

In [59]:
l.empty

True

In [57]:
v.empty

False

In [53]:
x = pd.concat(a,axis = 1).sum(axis=1, level=0)

In [54]:
x

,1,2
1,1.0,1.0
2,2.0,2.0
3,3.0,3.0
4,4.0,4.0
5,5.0,0.0


In [21]:
x.loc[x.index>2].sum(axis = 1)

3    6.0
4    8.0
5    5.0
6    5.0
dtype: float64

In [49]:
l = pd.to_datetime(pd.to_datetime("2007-06-20 13:50").strftime("%H:%M")) > pd.to_datetime("13:30")

In [50]:
l

True

In [43]:
z = pd.to_datetime(l.strftime("%H:%M"))

In [45]:
z

Timestamp('2022-04-18 13:50:00')

In [44]:
l > z

False

In [3]:
a = pd.DataFrame([1,2,3,4,5],index=[1,2,3,4,5])

In [25]:
a.std()

0    1.581139
dtype: float64

In [58]:
b = np.dot(a,a.T)

In [63]:
b

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

In [62]:
b-1

array([[ 0,  1,  2,  3,  4],
       [ 1,  3,  5,  7,  9],
       [ 2,  5,  8, 11, 14],
       [ 3,  7, 11, 15, 19],
       [ 4,  9, 14, 19, 24]])

In [61]:
b* (b-1)

array([[  0,   2,   6,  12,  20],
       [  2,  12,  30,  56,  90],
       [  6,  30,  72, 132, 210],
       [ 12,  56, 132, 240, 380],
       [ 20,  90, 210, 380, 600]])

In [59]:
b * b

array([[  1,   4,   9,  16,  25],
       [  4,  16,  36,  64, 100],
       [  9,  36,  81, 144, 225],
       [ 16,  64, 144, 256, 400],
       [ 25, 100, 225, 400, 625]])

In [8]:
pd.DataFrame(a)

ValueError: If using all scalar values, you must pass an index

In [26]:
temp = pd.read_csv("5_mins_mid_quote_ret.csv")
temp.head()

,Datetime,A,AA,AAPL,ABC,ABI,ABT,ACAS,ACE,ACS,...,XRX,XTO,YHOO,YUM,ZION,ZMH,COV,DFS,TEL,JAVA
0,2007-09-04 13:35:00,0.000276,-0.005647,0.012681,-0.003861,-0.001587,-0.002817,-0.014298,0.005729,0.000000,...,-0.001174,0.001476,0.008373,0.000000,0.001915,0.002798,0.000500,-0.002167,-0.006028,0.003721
1,2007-09-04 13:40:00,-0.002204,-0.002493,0.002963,-0.002304,-0.000795,0.000584,0.007867,0.005092,-0.001003,...,-0.000294,0.001842,0.002129,-0.002904,0.002122,0.000444,0.000000,-0.005430,-0.003754,0.001854
2,2007-09-04 13:45:00,0.000138,0.005139,-0.001547,-0.000105,0.007318,0.000097,0.003537,0.001117,-0.001306,...,0.002939,0.002207,0.000425,0.003066,0.000635,-0.003170,-0.000750,-0.000655,-0.004638,0.001850
3,2007-09-04 13:50:00,0.003451,-0.000829,0.000176,-0.002520,0.000158,0.001363,-0.000486,-0.004635,0.004928,...,0.000879,0.004128,0.000425,-0.001070,-0.001975,-0.003116,0.003503,0.004589,0.001748,0.009234
4,2007-09-04 13:55:00,-0.000275,-0.000830,-0.000458,0.000211,-0.003158,0.001750,-0.001702,-0.002414,0.003803,...,0.000293,0.003563,0.005519,-0.001683,-0.001767,-0.000319,-0.001620,-0.000653,0.003053,-0.003660
